In [1]:
import pandas as pd
import requests
import json
from pprint import pprint
from config import api_key
from census import Census
from us import states

In [2]:
# Setting up lists for looping and data retrieval
start_date = [
    "2020-12-31T00:00:00Z",
    "2021-01-31T00:00:00Z",
    "2021-02-28T00:00:00Z",
    "2021-03-31T00:00:00Z",
    "2021-04-30T00:00:00Z",
    "2021-05-31T00:00:00Z",
    "2021-06-30T00:00:00Z",
    "2021-07-31T00:00:00Z",
    "2021-08-31T00:00:00Z",
    "2021-09-30T00:00:00Z",
    "2021-10-31T00:00:00Z",
    "2021-11-30T00:00:00Z"
]

end_date = [
    "2021-02-01T00:00:00Z",
    "2021-03-01T00:00:00Z",
    "2021-04-01T00:00:00Z",
    "2021-05-01T00:00:00Z",
    "2021-06-01T00:00:00Z",
    "2021-07-01T00:00:00Z",
    "2021-08-01T00:00:00Z",
    "2021-09-01T00:00:00Z",
    "2021-10-01T00:00:00Z",
    "2021-11-01T00:00:00Z",
    "2021-12-01T00:00:00Z",
    "2022-01-01T00:00:00Z"
]

month = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec"
]

## Retrieve data from COVID19 API

Worth noting for any grader that may choose to execute the code again, it could take a while to execute the following calls to the COVID19 API. It has a tendency to time out and sometimes multiple attempts are needed. The code DOES work, it's not a problem with the code.

In [10]:
deaths_url = f"https://api.covid19api.com/country/united-states/status/deaths?from={start_date[0]}&to={end_date[11]}&province=California"
death_data = requests.get(deaths_url).json()
death_data

[{'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'Fresno',
  'CityCode': '6019',
  'Lat': '36.76',
  'Lon': '-119.65',
  'Cases': 711,
  'Status': 'deaths',
  'Date': '2020-12-31T00:00:00Z'},
 {'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'El Dorado',
  'CityCode': '6017',
  'Lat': '38.78',
  'Lon': '-120.52',
  'Cases': 26,
  'Status': 'deaths',
  'Date': '2020-12-31T00:00:00Z'},
 {'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'Mono',
  'CityCode': '6051',
  'Lat': '37.94',
  'Lon': '-118.89',
  'Cases': 3,
  'Status': 'deaths',
  'Date': '2020-12-31T00:00:00Z'},
 {'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'Yolo',
  'CityCode': '6113',
  'Lat': '38.68',
  'Lon': '-121.9',
  'Cases': 117,
  'Status': 'deaths',
  'Date': '2020-12-31T00:00:00Z'},
 {'Country': 'United Stat

In [12]:
cases_url = f"https://api.covid19api.com/country/united-states/status/confirmed?from={start_date[0]}&to={end_date[11]}&province=California"
case_data = requests.get(cases_url).json()
case_data

[{'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'Fresno',
  'CityCode': '6019',
  'Lat': '36.76',
  'Lon': '-119.65',
  'Cases': 71458,
  'Status': 'confirmed',
  'Date': '2020-12-31T00:00:00Z'},
 {'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'El Dorado',
  'CityCode': '6017',
  'Lat': '38.78',
  'Lon': '-120.52',
  'Cases': 6014,
  'Status': 'confirmed',
  'Date': '2020-12-31T00:00:00Z'},
 {'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'Mono',
  'CityCode': '6051',
  'Lat': '37.94',
  'Lon': '-118.89',
  'Cases': 880,
  'Status': 'confirmed',
  'Date': '2020-12-31T00:00:00Z'},
 {'Country': 'United States of America',
  'CountryCode': 'US',
  'Province': 'California',
  'City': 'Yolo',
  'CityCode': '6113',
  'Lat': '38.68',
  'Lon': '-121.9',
  'Cases': 8363,
  'Status': 'confirmed',
  'Date': '2020-12-31T00:00:00Z'},
 {'Cou

## Compile retrieved data, find cumulative deaths and cases at end of each month, calculate new cases each month and compile into a dataframe

In [13]:
# Convert Datasets to DataFrames
death_df = pd.DataFrame(death_data)
case_df = pd.DataFrame(case_data)

In [14]:
# Creating list of unique county names
county_finder_df = death_df
clean_county_finder_df = county_finder_df[county_finder_df["City"] != "Unassigned"]
clean_county_finder_df = clean_county_finder_df[clean_county_finder_df["CityCode"] != "80006"]
county_list = clean_county_finder_df["City"].unique()
county_list.sort()
county_list

array(['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne',
       'Ventura', 'Yolo', 'Yuba'], dtype=object)

In [15]:
# Create empty dataframe to hold all data
all_months_df = pd.DataFrame(columns=["County",
                                      "Cumulative Deaths by End of Month",
                                      "New Deaths During Month",
                                      "Cumulative Confirmed Cases by End of Month",
                                      "New Confirmed Cases During Month",
                                      "Month",
                                      "Year",
                                      "Latitude",
                                      "Longitude"]
                            )
all_months_df

,County,Cumulative Deaths by End of Month,New Deaths During Month,Cumulative Confirmed Cases by End of Month,New Confirmed Cases During Month,Month,Year,Latitude,Longitude


In [57]:
# Create index to tick up through all loops
index2 = 0

# Loop through all datasets
for index, date in enumerate(start_date):
    
    # Loop through dataframes
    for county in county_list:
        
        # Find cumulative deaths prior to target month, cumulative deaths at the end of the month and total deaths for the month
        county_prev_month_cum_deaths = int(death_df.loc[((death_df["City"] == county) & (death_df["Date"] == date)), "Cases"])
        county_month_cum_deaths = int(death_df.loc[((death_df["City"] == county) & (death_df["Date"] == end_date[index])), "Cases"])
        county_new_deaths = county_month_cum_deaths - county_prev_month_cum_deaths
        if county_new_deaths < 0:
            county_new_deaths = 0
        
        # Find cumulative cases prior to target month, cumulative cases at the end of the month and total cases for the month
        county_prev_month_cum_cases = int(case_df.loc[((case_df["City"] == county) & (case_df["Date"] == date)), "Cases"])
        county_month_cum_cases = int(case_df.loc[((case_df["City"] == county) & (case_df["Date"] == end_date[index])), "Cases"])
        county_new_cases = county_month_cum_cases - county_prev_month_cum_cases
        if county_new_cases < 0:
            county_new_cases = 0
        
        county_lat_list = death_df.loc[death_df["City"] == county, "Lat"].reset_index(drop=True)
        county_lat = county_lat_list[1]
        
        county_lon_list = death_df.loc[death_df["City"] == county, "Lon"].reset_index(drop=True)
        county_lon = county_lon_list[1]
                
        # Add values to month_df
        all_months_df.loc[index2, "County"] = county
        all_months_df.loc[index2, "Cumulative Deaths by End of Month"] = county_month_cum_deaths
        all_months_df.loc[index2, "New Deaths During Month"] = county_new_deaths
        all_months_df.loc[index2, "Cumulative Confirmed Cases by End of Month"] = county_month_cum_cases
        all_months_df.loc[index2, "New Confirmed Cases During Month"] = county_new_cases
        all_months_df.loc[index2, "Month"] = month[index]
        all_months_df.loc[index2, "Year"] = "2021"
        all_months_df.loc[index2, "Latitude"] = county_lat
        all_months_df.loc[index2, "Longitude"] = county_lon
        
        # Tick up index2
        index2 += 1
        
all_months_df

,County,Cumulative Deaths by End of Month,New Deaths During Month,Cumulative Confirmed Cases by End of Month,New Confirmed Cases During Month,Month,Year,Latitude,Longitude
0,Alameda,858,279,75947,22429,Jan,2021,37.65,-121.89
1,Alpine,0,0,74,7,Jan,2021,38.6,-119.82
2,Amador,32,8,3260,747,Jan,2021,38.45,-120.66
3,Butte,141,36,11789,3387,Jan,2021,39.67,-121.6
4,Calaveras,21,0,1796,632,Jan,2021,38.21,-120.55
...,...,...,...,...,...,...,...,...,...
691,Tulare,1160,64,89376,3590,Dec,2021,36.22,-118.8
692,Tuolumne,91,1,8170,413,Dec,2021,38.03,-119.95
693,Ventura,1211,21,113907,9924,Dec,2021,34.44,-119.09
694,Yolo,270,13,23281,1764,Dec,2021,38.68,-121.9


## Retrieve Census Data

In [28]:
c = Census(api_key)
census_data = c.acs5.get(
    (
        "NAME",                   #
        "B19013_001E",            # Median Income
        "B19301_001E",            # Income per capita
        "B17001_002E",            # poverty: Number of persons whose income in the past 12 months is below the poverty level
        "B01002_001E",            # Median Age
        "B01003_001E",            # Total Population
        "B02001_002E",            # White Population
        "B02001_003E",            # Black Population
        "B02001_004E",            # American Indian population
        "B02001_005E",            # Asian population
        "B02001_008E",            # Population identifying as 2+ races
        "B03001_003E",            # Hispanic population
        "B08301_010E"             # Percent of population using public transport
    ), geo={"for": "county:*",
            "in": "state:{}".format(states.CA.fips)}
)

census_pd = pd.DataFrame(census_data)
census_pd

,NAME,B19013_001E,B19301_001E,B17001_002E,B01002_001E,B01003_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_008E,B03001_003E,B08301_010E,state,county
0,"Alameda County, California",104888.0,49883.0,152060.0,37.8,1661584.0,631037.0,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0,06,001
1,"Alpine County, California",85750.0,37690.0,139.0,47.6,1159.0,663.0,10.0,360.0,6.0,111.0,183.0,4.0,06,003
2,"Butte County, California",54972.0,30700.0,39676.0,36.9,223344.0,178568.0,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0,06,007
3,"Colusa County, California",59427.0,27614.0,2540.0,35.4,21491.0,17012.0,330.0,239.0,201.0,2308.0,12840.0,13.0,06,011
4,"Contra Costa County, California",103997.0,50118.0,93021.0,39.9,1147788.0,608789.0,98569.0,5191.0,200159.0,104757.0,295791.0,56093.0,06,013
5,"El Dorado County, California",83710.0,44651.0,16018.0,46.3,190345.0,164552.0,1586.0,1312.0,9111.0,8659.0,24773.0,926.0,06,017
6,"Fresno County, California",57109.0,25757.0,202296.0,32.4,990204.0,597574.0,46182.0,11604.0,105302.0,84357.0,528293.0,3740.0,06,019
7,"Humboldt County, California",49235.0,29584.0,26210.0,38.9,136101.0,107151.0,1558.0,6162.0,3948.0,10757.0,16228.0,996.0,06,023
8,"Imperial County, California",46222.0,18064.0,38408.0,32.5,180580.0,102305.0,4716.0,2111.0,2599.0,19433.0,152880.0,435.0,06,025
9,"Kern County, California",54851.0,23855.0,175902.0,31.9,892458.0,603368.0,48530.0,8462.0,43381.0,75215.0,480700.0,2266.0,06,029


In [29]:
census_pd = census_pd.rename(
    columns = {
        "NAME": "County",
        "B19013_001E": "Median Income",
        "B19301_001E": "Income per Capita",
        "B17001_002E": "Number of Poeple in Poverty",
        "B01002_001E": "Median Age",
        "B01003_001E": "Total Population",
        "B02001_002E": "Total White Population",
        "B02001_003E": "Total Black Population",
        "B02001_004E": "Total American Indian Population",
        "B02001_005E": "Total Asian Poulation",
        "B02001_008E": "Total Population Identifying as 2+ Races",
        "B03001_003E": "Total Hispanic Population",
        "B08301_010E": "Total Pop Using Public Transit"
    }
)
census_pd

,County,Median Income,Income per Capita,Number of Poeple in Poverty,Median Age,Total Population,Total White Population,Total Black Population,Total American Indian Population,Total Asian Poulation,Total Population Identifying as 2+ Races,Total Hispanic Population,Total Pop Using Public Transit,state,county
0,"Alameda County, California",104888.0,49883.0,152060.0,37.8,1661584.0,631037.0,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0,06,001
1,"Alpine County, California",85750.0,37690.0,139.0,47.6,1159.0,663.0,10.0,360.0,6.0,111.0,183.0,4.0,06,003
2,"Butte County, California",54972.0,30700.0,39676.0,36.9,223344.0,178568.0,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0,06,007
3,"Colusa County, California",59427.0,27614.0,2540.0,35.4,21491.0,17012.0,330.0,239.0,201.0,2308.0,12840.0,13.0,06,011
4,"Contra Costa County, California",103997.0,50118.0,93021.0,39.9,1147788.0,608789.0,98569.0,5191.0,200159.0,104757.0,295791.0,56093.0,06,013
5,"El Dorado County, California",83710.0,44651.0,16018.0,46.3,190345.0,164552.0,1586.0,1312.0,9111.0,8659.0,24773.0,926.0,06,017
6,"Fresno County, California",57109.0,25757.0,202296.0,32.4,990204.0,597574.0,46182.0,11604.0,105302.0,84357.0,528293.0,3740.0,06,019
7,"Humboldt County, California",49235.0,29584.0,26210.0,38.9,136101.0,107151.0,1558.0,6162.0,3948.0,10757.0,16228.0,996.0,06,023
8,"Imperial County, California",46222.0,18064.0,38408.0,32.5,180580.0,102305.0,4716.0,2111.0,2599.0,19433.0,152880.0,435.0,06,025
9,"Kern County, California",54851.0,23855.0,175902.0,31.9,892458.0,603368.0,48530.0,8462.0,43381.0,75215.0,480700.0,2266.0,06,029


In [30]:
sorted_census_pd = census_pd.sort_values("County").reset_index(drop=True)
sorted_census_pd

,County,Median Income,Income per Capita,Number of Poeple in Poverty,Median Age,Total Population,Total White Population,Total Black Population,Total American Indian Population,Total Asian Poulation,Total Population Identifying as 2+ Races,Total Hispanic Population,Total Pop Using Public Transit,state,county
0,"Alameda County, California",104888.0,49883.0,152060.0,37.8,1661584.0,631037.0,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0,06,001
1,"Alpine County, California",85750.0,37690.0,139.0,47.6,1159.0,663.0,10.0,360.0,6.0,111.0,183.0,4.0,06,003
2,"Amador County, California",65187.0,33897.0,2872.0,49.9,39023.0,33040.0,931.0,276.0,375.0,2834.0,5591.0,35.0,06,005
3,"Butte County, California",54972.0,30700.0,39676.0,36.9,223344.0,178568.0,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0,06,007
4,"Calaveras County, California",67054.0,33027.0,5177.0,52.8,45828.0,40058.0,450.0,518.0,864.0,2912.0,5710.0,142.0,06,009
5,"Colusa County, California",59427.0,27614.0,2540.0,35.4,21491.0,17012.0,330.0,239.0,201.0,2308.0,12840.0,13.0,06,011
6,"Contra Costa County, California",103997.0,50118.0,93021.0,39.9,1147788.0,608789.0,98569.0,5191.0,200159.0,104757.0,295791.0,56093.0,06,013
7,"Del Norte County, California",49981.0,24361.0,4223.0,40.0,27692.0,20002.0,747.0,1943.0,880.0,2160.0,5552.0,31.0,06,015
8,"El Dorado County, California",83710.0,44651.0,16018.0,46.3,190345.0,164552.0,1586.0,1312.0,9111.0,8659.0,24773.0,926.0,06,017
9,"Fresno County, California",57109.0,25757.0,202296.0,32.4,990204.0,597574.0,46182.0,11604.0,105302.0,84357.0,528293.0,3740.0,06,019


In [31]:
sorted_census_pd["County"] = county_list
sorted_census_pd.head()

,County,Median Income,Income per Capita,Number of Poeple in Poverty,Median Age,Total Population,Total White Population,Total Black Population,Total American Indian Population,Total Asian Poulation,Total Population Identifying as 2+ Races,Total Hispanic Population,Total Pop Using Public Transit,state,county
0,Alameda,104888.0,49883.0,152060.0,37.8,1661584.0,631037.0,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0,06,001
1,Alpine,85750.0,37690.0,139.0,47.6,1159.0,663.0,10.0,360.0,6.0,111.0,183.0,4.0,06,003
2,Amador,65187.0,33897.0,2872.0,49.9,39023.0,33040.0,931.0,276.0,375.0,2834.0,5591.0,35.0,06,005
3,Butte,54972.0,30700.0,39676.0,36.9,223344.0,178568.0,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0,06,007
4,Calaveras,67054.0,33027.0,5177.0,52.8,45828.0,40058.0,450.0,518.0,864.0,2912.0,5710.0,142.0,06,009


In [32]:
del sorted_census_pd["state"]
del sorted_census_pd["county"]
sorted_census_pd.head()

,County,Median Income,Income per Capita,Number of Poeple in Poverty,Median Age,Total Population,Total White Population,Total Black Population,Total American Indian Population,Total Asian Poulation,Total Population Identifying as 2+ Races,Total Hispanic Population,Total Pop Using Public Transit
0,Alameda,104888.0,49883.0,152060.0,37.8,1661584.0,631037.0,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0
1,Alpine,85750.0,37690.0,139.0,47.6,1159.0,663.0,10.0,360.0,6.0,111.0,183.0,4.0
2,Amador,65187.0,33897.0,2872.0,49.9,39023.0,33040.0,931.0,276.0,375.0,2834.0,5591.0,35.0
3,Butte,54972.0,30700.0,39676.0,36.9,223344.0,178568.0,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0
4,Calaveras,67054.0,33027.0,5177.0,52.8,45828.0,40058.0,450.0,518.0,864.0,2912.0,5710.0,142.0


## Pull county data from world population review website csv

In [47]:
county_data = pd.read_csv("Resources/county_data.csv")
county_data

,year,fips,pop2010,pop2023,state,stateCode,growthSince2010,name,ctyname,slug,landArea,density,StatePercentage,StateRank,densityMi,area
0,2023,6037,9818605,10072629,California,CA,0.025872,Los Angeles County,Los Angeles County,los-angeles-county-ca,4058,2482.165845,0.250417,1,2482.165845,4058
1,2023,6073,3095313,3359630,California,CA,0.085393,San Diego County,San Diego County,san-diego-county-ca,4207,798.580937,0.083524,2,798.580937,4207
2,2023,6059,3010232,3240017,California,CA,0.076335,Orange County,Orange County,orange-county-ca,791,4096.102402,0.080550,3,4096.102402,791
3,2023,6065,2189641,2486747,California,CA,0.135687,Riverside County,Riverside County,riverside-county-ca,7206,345.093949,0.061823,4,345.093949,7206
4,2023,6071,2035210,2225586,California,CA,0.093541,San Bernardino County,San Bernardino County,san-bernardino-county-ca,20057,110.963055,0.055330,5,110.963055,20057
5,2023,6085,1781642,1982645,California,CA,0.112819,Santa Clara County,Santa Clara County,santa-clara-county-ca,1290,1536.934109,0.049291,6,1536.934109,1290
6,2023,6001,1510271,1733977,California,CA,0.148123,Alameda County,Alameda County,alameda-county-ca,739,2346.382950,0.043109,7,2346.382950,739
7,2023,6067,1418788,1634936,California,CA,0.152347,Sacramento County,Sacramento County,sacramento-county-ca,965,1694.234197,0.040646,8,1694.234197,965
8,2023,6013,1049025,1200997,California,CA,0.144870,Contra Costa County,Contra Costa County,contra-costa-county-ca,723,1661.130014,0.029858,9,1661.130014,723
9,2023,6019,930450,1032114,California,CA,0.109263,Fresno County,Fresno County,fresno-county-ca,5959,173.202551,0.025659,10,173.202551,5959


In [48]:
sorted_county_data = county_data.sort_values("name").reset_index(drop=True)
sorted_county_data

,year,fips,pop2010,pop2023,state,stateCode,growthSince2010,name,ctyname,slug,landArea,density,StatePercentage,StateRank,densityMi,area
0,2023,6001,1510271,1733977,California,CA,0.148123,Alameda County,Alameda County,alameda-county-ca,739,2346.382950,0.043109,7,2346.382950,739
1,2023,6003,1175,1213,California,CA,0.032340,Alpine County,Alpine County,alpine-county-ca,738,1.643631,0.000030,58,1.643631,738
2,2023,6005,38091,41188,California,CA,0.081305,Amador County,Amador County,amador-county-ca,595,69.223529,0.001024,46,69.223529,595
3,2023,6007,220000,209121,California,CA,-0.049450,Butte County,Butte County,butte-county-ca,1636,127.824572,0.005199,28,127.824572,1636
4,2023,6009,45578,45205,California,CA,-0.008184,Calaveras County,Calaveras County,calaveras-county-ca,1020,44.318627,0.001124,44,44.318627,1020
5,2023,6011,21419,21965,California,CA,0.025491,Colusa County,Colusa County,colusa-county-ca,1151,19.083406,0.000546,50,19.083406,1151
6,2023,6013,1049025,1200997,California,CA,0.144870,Contra Costa County,Contra Costa County,contra-costa-county-ca,723,1661.130014,0.029858,9,1661.130014,723
7,2023,6015,28610,27482,California,CA,-0.039427,Del Norte County,Del Norte County,del-norte-county-ca,1006,27.318091,0.000683,49,27.318091,1006
8,2023,6017,181058,194224,California,CA,0.072717,El Dorado County,El Dorado County,el-dorado-county-ca,1708,113.714286,0.004829,29,113.714286,1708
9,2023,6019,930450,1032114,California,CA,0.109263,Fresno County,Fresno County,fresno-county-ca,5959,173.202551,0.025659,10,173.202551,5959


In [49]:
sorted_county_data = sorted_county_data.rename(
    columns = {
        "name": "County",
        "landArea": "Land Area (mi^2)",
    }
)

del sorted_county_data["year"]
del sorted_county_data["fips"]
del sorted_county_data["pop2010"]
del sorted_county_data["pop2023"]
del sorted_county_data["state"]
del sorted_county_data["stateCode"]
del sorted_county_data["growthSince2010"]
del sorted_county_data["ctyname"]
del sorted_county_data["slug"]
del sorted_county_data["density"]
del sorted_county_data["StatePercentage"]
del sorted_county_data["StateRank"]
del sorted_county_data["densityMi"]
del sorted_county_data["area"]

sorted_county_data["County"] = county_list

sorted_county_data

,County,Land Area (mi^2)
0,Alameda,739
1,Alpine,738
2,Amador,595
3,Butte,1636
4,Calaveras,1020
5,Colusa,1151
6,Contra Costa,723
7,Del Norte,1006
8,El Dorado,1708
9,Fresno,5959


In [58]:
census_covid_combined_data = pd.merge(all_months_df, sorted_census_pd, on="County", how="left")
census_covid_combined_data

,County,Cumulative Deaths by End of Month,New Deaths During Month,Cumulative Confirmed Cases by End of Month,New Confirmed Cases During Month,Month,Year,Latitude,Longitude,Median Income,...,Number of Poeple in Poverty,Median Age,Total Population,Total White Population,Total Black Population,Total American Indian Population,Total Asian Poulation,Total Population Identifying as 2+ Races,Total Hispanic Population,Total Pop Using Public Transit
0,Alameda,858,279,75947,22429,Jan,2021,37.65,-121.89,104888.0,...,152060.0,37.8,1661584.0,631037.0,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0
1,Alpine,0,0,74,7,Jan,2021,38.6,-119.82,85750.0,...,139.0,47.6,1159.0,663.0,10.0,360.0,6.0,111.0,183.0,4.0
2,Amador,32,8,3260,747,Jan,2021,38.45,-120.66,65187.0,...,2872.0,49.9,39023.0,33040.0,931.0,276.0,375.0,2834.0,5591.0,35.0
3,Butte,141,36,11789,3387,Jan,2021,39.67,-121.6,54972.0,...,39676.0,36.9,223344.0,178568.0,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0
4,Calaveras,21,0,1796,632,Jan,2021,38.21,-120.55,67054.0,...,5177.0,52.8,45828.0,40058.0,450.0,518.0,864.0,2912.0,5710.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,Tulare,1160,64,89376,3590,Dec,2021,36.22,-118.8,52534.0,...,99610.0,31.2,463955.0,309798.0,7606.0,5955.0,17310.0,37160.0,301919.0,1120.0
692,Tuolumne,91,1,8170,413,Dec,2021,38.03,-119.95,60509.0,...,5564.0,48.7,54147.0,46905.0,1007.0,948.0,753.0,2789.0,6810.0,91.0
693,Ventura,1211,21,113907,9924,Dec,2021,34.44,-119.09,89295.0,...,74449.0,38.5,845599.0,637686.0,15422.0,6990.0,61239.0,71894.0,361648.0,3998.0
694,Yolo,270,13,23281,1764,Dec,2021,38.68,-121.9,73746.0,...,39197.0,31.1,218774.0,145299.0,5694.0,1420.0,31584.0,21326.0,69341.0,4097.0


In [59]:
combined_data = pd.merge(census_covid_combined_data, sorted_county_data, on="County", how="left")
combined_data

,County,Cumulative Deaths by End of Month,New Deaths During Month,Cumulative Confirmed Cases by End of Month,New Confirmed Cases During Month,Month,Year,Latitude,Longitude,Median Income,...,Median Age,Total Population,Total White Population,Total Black Population,Total American Indian Population,Total Asian Poulation,Total Population Identifying as 2+ Races,Total Hispanic Population,Total Pop Using Public Transit,Land Area (mi^2)
0,Alameda,858,279,75947,22429,Jan,2021,37.65,-121.89,104888.0,...,37.8,1661584.0,631037.0,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0,739
1,Alpine,0,0,74,7,Jan,2021,38.6,-119.82,85750.0,...,47.6,1159.0,663.0,10.0,360.0,6.0,111.0,183.0,4.0,738
2,Amador,32,8,3260,747,Jan,2021,38.45,-120.66,65187.0,...,49.9,39023.0,33040.0,931.0,276.0,375.0,2834.0,5591.0,35.0,595
3,Butte,141,36,11789,3387,Jan,2021,39.67,-121.6,54972.0,...,36.9,223344.0,178568.0,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0,1636
4,Calaveras,21,0,1796,632,Jan,2021,38.21,-120.55,67054.0,...,52.8,45828.0,40058.0,450.0,518.0,864.0,2912.0,5710.0,142.0,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,Tulare,1160,64,89376,3590,Dec,2021,36.22,-118.8,52534.0,...,31.2,463955.0,309798.0,7606.0,5955.0,17310.0,37160.0,301919.0,1120.0,4824
692,Tuolumne,91,1,8170,413,Dec,2021,38.03,-119.95,60509.0,...,48.7,54147.0,46905.0,1007.0,948.0,753.0,2789.0,6810.0,91.0,2221
693,Ventura,1211,21,113907,9924,Dec,2021,34.44,-119.09,89295.0,...,38.5,845599.0,637686.0,15422.0,6990.0,61239.0,71894.0,361648.0,3998.0,1843
694,Yolo,270,13,23281,1764,Dec,2021,38.68,-121.9,73746.0,...,31.1,218774.0,145299.0,5694.0,1420.0,31584.0,21326.0,69341.0,4097.0,1015


In [60]:
combined_data["Population Density (persons/mi^2)"] = combined_data["Total Population"]/combined_data["Land Area (mi^2)"]
combined_data["New Cases/ 100k People"] = combined_data["New Confirmed Cases During Month"]/combined_data["Total Population"] * 100000
combined_data["New Deaths/ 100k People"] = combined_data["New Deaths During Month"]/combined_data["Total Population"] * 100000

combined_data

,County,Cumulative Deaths by End of Month,New Deaths During Month,Cumulative Confirmed Cases by End of Month,New Confirmed Cases During Month,Month,Year,Latitude,Longitude,Median Income,...,Total Black Population,Total American Indian Population,Total Asian Poulation,Total Population Identifying as 2+ Races,Total Hispanic Population,Total Pop Using Public Transit,Land Area (mi^2),Population Density (persons/mi^2),New Cases/ 100k People,New Deaths/ 100k People
0,Alameda,858,279,75947,22429,Jan,2021,37.65,-121.89,104888.0,...,171922.0,11037.0,519249.0,130496.0,369546.0,114029.0,739,2248.422192,1349.86,16.7912
1,Alpine,0,0,74,7,Jan,2021,38.6,-119.82,85750.0,...,10.0,360.0,6.0,111.0,183.0,4.0,738,1.570461,603.969,0
2,Amador,32,8,3260,747,Jan,2021,38.45,-120.66,65187.0,...,931.0,276.0,375.0,2834.0,5591.0,35.0,595,65.584874,1914.26,20.5007
3,Butte,141,36,11789,3387,Jan,2021,39.67,-121.6,54972.0,...,3866.0,2628.0,10743.0,15608.0,37585.0,1010.0,1636,136.518337,1516.49,16.1186
4,Calaveras,21,0,1796,632,Jan,2021,38.21,-120.55,67054.0,...,450.0,518.0,864.0,2912.0,5710.0,142.0,1020,44.929412,1379.07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,Tulare,1160,64,89376,3590,Dec,2021,36.22,-118.8,52534.0,...,7606.0,5955.0,17310.0,37160.0,301919.0,1120.0,4824,96.176410,773.782,13.7944
692,Tuolumne,91,1,8170,413,Dec,2021,38.03,-119.95,60509.0,...,1007.0,948.0,753.0,2789.0,6810.0,91.0,2221,24.379559,762.738,1.84682
693,Ventura,1211,21,113907,9924,Dec,2021,34.44,-119.09,89295.0,...,15422.0,6990.0,61239.0,71894.0,361648.0,3998.0,1843,458.816603,1173.61,2.48345
694,Yolo,270,13,23281,1764,Dec,2021,38.68,-121.9,73746.0,...,5694.0,1420.0,31584.0,21326.0,69341.0,4097.0,1015,215.540887,806.312,5.94221


In [61]:
combined_data.to_csv("Resources/combined_covid_census_data.csv", index=False)